In [1]:
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
import pandas as pd
import os
import re

In [2]:
# Helper function
def clean_rest_string(input_str):
    # Define a regex pattern to remove the hyphen "—" and the newline character "\n"
    pattern = r'[—\n\.]'
    # Use the re.sub() function to replace the matched pattern with an empty string
    output_str = re.sub(pattern, ' ', input_str)
    output_str = re.sub(r'\s+', ' ', output_str)
    return(output_str)

In [7]:
# Convert to JPG from source
images = convert_from_path('../data/PARIS_1867_Italy.pdf')

for count, image in enumerate(images, start=first_page):
    image.save(f'../data/Paris1867_Italy_{count}.jpg', 'JPEG')

In [3]:
# Define the folder path where the image files are located
folder_path = '../data'

# Initialize an empty dictionary to store the text for each page
output_dict = {}

# Loop through all the .jpg files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") and "Paris1867_Italy" in filename:
        # Create the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Extract text from the image using pytesseract
        text = pytesseract.image_to_string(Image.open(file_path), lang='fra')
        
        # Add the text to the dictionary with the filename as the key
        output_dict[filename] = text

In [4]:
# Initialize an empty list to store the split text and sources
split_text_list = []
source_list = []

# Iterate through the elements in the output_dict
for filename, text in output_dict.items():
    # Split the text by '\n\n' and add the resulting elements to the list
    split_entries = text.split('\n\n')
    
    # Add each split entry to the split_text_list and its corresponding source to the source_list
    for entry in split_entries:
        split_text_list.append(entry)
        source_list.append(filename)

# Create a pandas DataFrame with 'raw_text' and 'source' columns
df = pd.DataFrame({'raw_text': split_text_list, 'source': source_list})


In [36]:
# Assuming you have your DataFrame df as described
# Create a list to store the filtered data
filtered_data = []

# Initialize a variable to track the 'class' value
current_class = None

# Iterate through the rows of the original DataFrame
for index, row in df.iterrows():
    raw_text = row['raw_text']
    source = row['source']
    # Check if the row contains the word "CLASSE"
    if "CLASSE" in raw_text:
        current_class = raw_text
    else:
        # If the row does not contain "CLASSE," add it to the list
        filtered_data.append({'raw_text': raw_text, 'class': current_class, 'source':source})

# Create a DataFrame from the list of dictionaries
filtered_df = pd.DataFrame(filtered_data)

By now, we have OCR'ed all the data. In what follows, we think that some of the line split ups have been done erroneously. Now we correct that on the basis of numbers. Thus, we decrease the number of lines to something closer to the actual number of lines. 

In [51]:
merged_text = []  # List to store merged text

for index, row in df.iterrows():
    if not any(char.isdigit() for char in row['raw_text'][:3]) and index > 0:
        # Merge the current row's text into the previous row
        merged_text[-1] += ' ' + row['raw_text']
    else:
        # Append the current row's text as a new entry
        merged_text.append(row['raw_text'])

In what follows, we simply rid the text of some annoying symbols without compromising a lot on the structure. Then afterwards, in process_lines, we cut up (increase) the number of lines again, on the basis of numbers, because we felt that some observations were erroneously clustered in 1 line. What results is a very nice and neat vector of raw strings, which should be easy to process. That is the purpose the everything afterwards. 

In [96]:
raw_text = (pd.Series(merged_text).str.replace(r'[^\w\s()&]', ' ').
                                   str.replace(r'\s+', ' ').
                                   str.replace(r'-', '').
                                   str.replace('\n', '').
                                   str.replace('', '').
                                   str.strip())
#).to_csv('../data/test2.csv')

In [97]:
def process_lines(lines):
    for line in lines:
        count = 0  # Counter for sequences of consecutive digits
        for match in re.finditer(r'\d+', line):
            count += 1
            if count == 2:
                split_index = match.start()  # Get the index of the second sequence of digits
                yield line[:split_index]  # Yield the part before the second sequence of digits
                yield from process_lines([line[split_index:]])  # Recursively process the remaining part
                break
        else:
            yield line  # If no split occurs, yield the original line

In [98]:
processed_lines = list(process_lines(raw_text))

In [104]:
raw_text_df = pd.DataFrame(pd.Series(processed_lines), columns=['raw_text'])

In [118]:
def extract_location(text):
    # Pattern to match word after "à" followed by space and potentially words in brackets
    pattern_a_accent = r"(?<=à\s)(\w+)(?:\s*\((.*?)\))?"
    
    # Pattern to match word after accent aigu without space and potentially words in brackets
    pattern_accent_aigu_no_space = r"(?<=à)(\w+)(?:\s*\((.*?)\))?|\Z"

    # Pattern to match word after "de" followed by space and potentially words in brackets
    pattern_de = r"(?<=de\s)(\w+)(?:\s*\((.*?)\))?"

    # Pattern to match word after "a" followed by space and potentially words in brackets
    pattern_a = r"(?<=a\s)(\w+)(?:\s*\((.*?)\))?"

    # First, try to match word after "à" followed by space and potentially words in brackets
    match = re.search(pattern_a_accent, text)
    if match:
        return match.group(1), match.group(2)
    
    # If nothing is found, try to match word after accent aigu without space and potentially words in brackets
    match = re.search(pattern_accent_aigu_no_space, text)
    if match:
        return match.group(1), match.group(2)

    # If not found, try to match word after "de" followed by space and potentially words in brackets
    match = re.search(pattern_de, text)
    if match:
        return match.group(1), match.group(2)

    # If still not found, try to match word after "a" followed by space and potentially words in brackets
    match = re.search(pattern_a, text)
    if match:
        return match.group(1), match.group(2)


    # If no matches found, return None
    return None, None



In [119]:
# Parse the location:
raw_text_df['location'] = raw_text_df['raw_text'].apply(lambda x: extract_location(x)[0])


In [ ]:
# Export the final resulting file to .csv
# And aftewards continue with name_matching.ipynb
raw_text_df.to_csv('../data/1867_italy.csv')

In [120]:
raw_text_df.to_csv('../data/test.csv')